In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

In [7]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

In [9]:
response.choices[0].message.content

"The answer to whether it's too late to join a course depends on several factors, including the specific course's registration deadlines, the policies of the institution offering the course, and how far along the course is. Here are some steps you can take to find out:\n\n1. **Check the Course Website**: The course's official website or the institution's course catalog will often have information about registration deadlines and add/drop periods.\n\n2. **Contact the Instructor or Advisor**: Reach out to the course instructor or an academic advisor. They can provide information on whether it's still possible to join and what steps you need to take if it's permitted.\n\n3. **Institution Policies**: Some institutions have late registration periods or allow students to add courses during the first week or two of the term. Check your institution's policy on late enrollment.\n\n4. **Self-Paced Courses**: If the course is self-paced, it may have more flexible enrollment deadlines, allowing yo

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [3]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█|


In [50]:
query = "How do I execute a command in a running docker container?"

def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

search_results = elastic_search(query)
for res in search_results:
    print(res)

{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}
{'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'mac

In [51]:
context_template = """
Q: {question}
A: {text}
""".strip()

def build_context(documents, query):
    context = ""
    for doc in documents:
        context += context_template.format(question=query, text=doc['text'])
        context += "\n\n"
    return context.strip()

records = elastic_search(query)

context = build_context(search_results, query)
print(context)

Q: How do I execute a command in a running docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I execute a command in a running docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I execute a command in a running docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In t

In [52]:
def build_prompt(query, context):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the question.

QUESTION: {question}

CONTEXT: {context}  
""".strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt;

In [53]:
prompt = build_prompt(query, context)
len(prompt)

1451

In [54]:
import tiktoken

In [55]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [56]:
tokens = encoding.encode(prompt)
len(tokens)

321

In [57]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

In [60]:
print(response.choices[0].message.content)

To execute a command in a running Docker container, you can follow these steps:

1. Identify the container ID of the running container using:
   ```
   docker ps
   ```
2. Execute a command within the specified container using:
   ```
   docker exec -it <container-id> bash
   ```

This method allows you to start a bash session within the running container where you can execute commands as needed.
